In [1]:
from utils.chat_model import init_chat_model
from utils.sound_script import read_script

llm = init_chat_model(dotenv_path="../.env")

# 大语言模型结构化输出 with langchain

(feat Bronya VITS)

大家好，今天分享一下使用langchain做大语言模型的结构化输出的一点经验

除了与人类进行聊天，有两个场景非常适合语言模型：

一是自主执行任务，比如我可以告诉GPT家里的温度和湿度，让他来帮我控制空调

二是进行内容创作，比如我们下面给出的这个例子，让语言模型帮我们设计一位随机角色：

In [2]:
result = llm.call_as_llm("你好，请帮我虚构一个游戏角色，游戏背景是科幻风格，发生在宇宙中。请给出他的姓名、种族、年龄、外貌、性别")
 
print(result)

姓名：艾丽斯·凯恩（Alice Kane）
种族：人类
年龄：27岁
外貌：身材高挑，长发披肩，蓝色的眼睛和红色的嘴唇，穿着黑色的紧身衣和金属盔甲，手持一把能量剑。
性别：女性

艾丽斯·凯恩是一名年轻的宇宙战士，她在年轻时就加入了银河联盟的军队。她有着敏锐的战术意识和出色的战斗技巧，她的能量剑能够轻易地切割金属和能量盾。在她的旅程中，她遇到了许多敌人和挑战，但她总是勇往直前，保护着她所在的星球和银河系。她的勇气和决心让她成为了银河联盟中最受尊敬的指挥官之一。


## 定义数据模型

我们确实得到一位人物，但是在代码里，无法直接使用这段文本来生成一个NPC。显然，语言模型被定义为使用人类的自然语言进行输出，但是我们现在需要他能够输出程序语言

如果能做到这一点，我们就可以用语言模型控制其他的程序，也就是GPT的插件功能。

目前普遍在使用的方法是，在指令中与语言模型进行输出格式的约定，对输出结果基于这个格式进行解析，我们来看一个示例：

In [3]:
import json
from enum import Enum

from IPython.display import Markdown as md
from pydantic import BaseModel, Field

class Gender(Enum):
    Male = "M"
    Female = "F"
    OTHER = "O"
    NA = ""

class Character(BaseModel):
    name: str = Field(description="姓名")
    species: str = Field(description="种族")
    age: int = Field(description="年龄")
    appearance: str = Field(description="外貌")
    gender: Gender = Field(description="性别,枚举")

## 格式化Prompt

首先定义好我们想要的数据结构，一个科幻游戏中的人物，它具有姓名，种族，年龄，性别四种属性和一段外貌描述文本。

接下来，我们基于这个数据结构来构造与语言模型的约定，在信息领域，人类发明了很多种用于描述数据结构的专用语言。langchain使用了 JSON 格式定义语法来表达数据结构，这是GPT比较熟悉的一种格式

通过使用`PydanticOutputParser`这个类，我们可以根据数据结构直接生成相应的Prompt

我们把这段说明和刚才的人物生成要求拼接到一起，并补充了对生成内容语言的要求

In [4]:
from langchain.output_parsers import PydanticOutputParser

parser = PydanticOutputParser(pydantic_object=Character)

md(f"""
**这是由定义的数据对象得到的`Json Schema`：**
```json
{json.dumps(Character.schema(),ensure_ascii=False)}
```

**Langchain内置的格式化指示Prompt:**

{parser.get_format_instructions()}
""")


**这是由定义的数据对象得到的`Json Schema`：**
```json
{"title": "Character", "type": "object", "properties": {"name": {"title": "Name", "description": "姓名", "type": "string"}, "species": {"title": "Species", "description": "种族", "type": "string"}, "age": {"title": "Age", "description": "年龄", "type": "integer"}, "appearance": {"title": "Appearance", "description": "外貌", "type": "string"}, "gender": {"description": "性别,枚举", "allOf": [{"$ref": "#/definitions/Gender"}]}}, "required": ["name", "species", "age", "appearance", "gender"], "definitions": {"Gender": {"title": "Gender", "description": "An enumeration.", "enum": ["M", "F", "O", ""]}}}
```

**Langchain内置的格式化指示Prompt:**

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"name": {"title": "Name", "description": "\u59d3\u540d", "type": "string"}, "species": {"title": "Species", "description": "\u79cd\u65cf", "type": "string"}, "age": {"title": "Age", "description": "\u5e74\u9f84", "type": "integer"}, "appearance": {"title": "Appearance", "description": "\u5916\u8c8c", "type": "string"}, "gender": {"description": "\u6027\u522b,\u679a\u4e3e", "allOf": [{"$ref": "#/definitions/Gender"}]}}, "required": ["name", "species", "age", "appearance", "gender"], "definitions": {"Gender": {"title": "Gender", "description": "An enumeration.", "enum": ["M", "F", "O", ""]}}}
```


In [5]:
from langchain import LLMChain, PromptTemplate


template = """
你好，请帮我虚构一个游戏角色，游戏背景是科幻风格，发生在宇宙中。请给出他的姓名、种族、年龄、外貌、性别

{format_instructions}
请注意，角色的描述需要使用中文
"""

prompt = PromptTemplate(
    template=template,
    input_variables=[],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = LLMChain(llm=llm, prompt=prompt)

result = chain({})

print(result["text"])
character:Character = parser.parse(result["text"])

md(f"""
|name|species|age|gender|appearance|
|---|---|---|---|---|
|{character.name}|{character.species}|{character.age}|{character.gender}|{character.appearance}|
""")

{
  "name": "艾丽丝",
  "species": "人类",
  "age": 28,
  "appearance": "身材高挑，黑色长直发，绿色眼睛，左耳上有个银色耳环",
  "gender": "F"
}



|name|species|age|gender|appearance|
|---|---|---|---|---|
|艾丽丝|人类|28|Gender.Female|身材高挑，黑色长直发，绿色眼睛，左耳上有个银色耳环|


## 修复错误的返回

有些时候，语言模型可能不会遵循你的格式要求，我把这种情况称之为**任性**

如果出现了模型任性的情况，可以再追加一次专门的对话，要求语言模型修复他之前返回的内容。请参考如下示例

In [18]:
misformat1 = """{
    "name": "李田所",
    "species": "人类",
    "年龄": "24岁",
    "外貌": "俊朗",
}"""

misformat2 = """
孙桑,40岁,僵尸,儒雅随和
"""


In [21]:
from langchain.output_parsers import OutputFixingParser

fixing_parser = OutputFixingParser.from_llm(parser=parser, llm=llm)


我们人为制造了一些格式异常的返回结果，这些结果无法被正确的解析为之前定义的对象

并使用langchain中内置的格式修复解析器，结合之前定义的Prompt，生成要求语言模型修复返回结果格式的指示文本

In [19]:
character = fixing_parser.parse(misformat1)

print(character.json(indent=4,ensure_ascii=False))

{
    "name": "李田所",
    "species": "人类",
    "age": 24,
    "appearance": "俊朗",
    "gender": "M"
}


In [16]:
character = fixing_parser.parse(misformat2)

print(character.json(indent=4,ensure_ascii=False))

{
    "name": "孙桑",
    "species": "僵尸",
    "age": 40,
    "appearance": "儒雅随和",
    "gender": ""
}


看，我们的异常内容都得到了修复 :)

## 关于Parser的一些个人使用经验

1. 定义数据模型时，变量的命名很重要，因为他们都会被转化成文本输出给语言模型。起名字和写注释的经验基本可以同步到这里，当做是让新来的程序员接受你的代码就可以

2. 如果你的任务比较复杂，需要语言模型先做思考或是推理，再返回结构化的结果，那么最好拆分为两次对话。否则这两个过程的效果可能会互相影响。比如我们先说出室内的温度、湿度和空调的状态，让语言模型帮我们决定是否要给空调下发指令，下发什么指令。再说明空调的调用接口，让语言模型生成格式化的空调调用参数

3. 根据我的个人经验，大家看到上面的格式化prompt都是英文的，没有做调整，这样做有时候会导致语言模型返回英文的结果
如果切换成中文的格式化说明的话，语言模型任性的概率会提升。可能更好的方式是全部使用英文，得到结果后再做翻译也说不定

4. 进行格式化输出的输入prompt越长，语言模型任性并且无法修复的概率都会随之提升，这个可能是和Token占比有关系，
因此，对很长的输入，还是尽量想办法拆分或是概括后再输入给语言模型。我也尝试过将格式化的说明文字复制多份，但是效果不显著。

## 简单看一下Parser相关的代码

In [10]:
from langchain.output_parsers import PydanticOutputParser, OutputFixingParser